In [2]:
from scipy.stats import beta
import pandas as pd
import lxml

In [3]:
df = pd.read_html('https://www.wahlrecht.de/umfragen/landtage/bremen.htm')

In [5]:
df[1][['CDU', 'SPD', 'GRÜNE', 'FDP', 'LINKE',  'AfD',  'Sonstige']].iloc[0]

CDU         22 %
SPD         30 %
GRÜNE       21 %
FDP          6 %
LINKE        8 %
AfD          6 %
Sonstige     7 %
Name: 0, dtype: object

In [7]:
def make_dist(mean, std):
    a = mean * ( mean**2/std**2 * (1 /mean - 1) -1)
    b = a * (1/mean - 1)

    return beta(a, b)

In [11]:
cdu = make_dist(0.22, 0.02)
spd = make_dist(0.30, 0.02)
gruen = make_dist(0.21, 0.02)
fdp = make_dist(0.06, 0.02)
linke = make_dist(0.07, 0.02)
afd = make_dist(0.06, 0.02)

parties = {
    'cdu': cdu, 
    'spd': spd, 
    'gruen': gruen, 
    'fdp': fdp, 
    'linke': linke, 
    'afd': afd
}

def get_prediction():
    result = {}
    total = 0
    for name, p in parties.items():
        result[name] = p.rvs(1)[0]
        total += result[name]
    result = {name: p / total for name, p in result.items()}
    return result

def predictions():
    for _ in range(1000):
        yield get_prediction()

get_prediction()

{'cdu': 0.23346191662134225,
 'spd': 0.29581385735620763,
 'gruen': 0.2156240167483099,
 'fdp': 0.07713723598539098,
 'linke': 0.09273472066497672,
 'afd': 0.08522825262377251}

In [14]:
import random

def is_possible(prediction, coalition):
    if sum(prediction[c] for c in coalition) > 0.5:
        return True

    return False

def choose_result(result):
    if "spd_gruen" in result:
        return "spd_gruen"

    if "spd_gruen_fdp" in result and "spd_cdu" in result:
        if random.random() > 0.5:
            return "spd_gruen_fdp"
        else:
            return "spd_cdu"

    return result[0]

coalitions = [
    ["spd", "gruen"],
    ["spd", "cdu"],
    ["spd", "gruen", "fdp"],
    ["cdu", "afd"]
]

results = []

for prediction in predictions():
    results.append([ "_".join(c) for c in coalitions if is_possible(prediction, c)])

from collections import Counter

Counter([choose_result(r) for r in results])

Counter({'spd_gruen': 978, 'spd_cdu': 13, 'spd_gruen_fdp': 9})

In [13]:
party_names = parties.keys()
passed_5 = {name: 0 for name in party_names}

for prediction in predictions():
    for name in party_names:
        if prediction[name] > 0.05:
            passed_5[name] += 1
    
passed_5

{'cdu': 1000, 'spd': 1000, 'gruen': 1000, 'fdp': 752, 'linke': 911, 'afd': 755}

## Bayern

In [16]:
df = pd.read_html('https://www.wahlrecht.de/umfragen/landtage/bayern.htm')[1]

df.head()

,Institut,Auftraggeber,Befragte,Datum,Unnamed: 4,CSU,SPD,GRÜNE,FDP,LINKE,FW,AfD,Sonstige
0,GMS,Sat.1 Bayern,T • 1.004 14.09.–20.09.,21.09.2022,NaN,40 %,8 %,18 %,6 %,2 %,10 %,11 %,BP 1 % Sonst. 4 %
1,INSA,BILD,O • 1.000 20.06.–27.06.,29.06.2022,NaN,37 %,10 %,20 %,7 %,2 %,10 %,9 %,5 %
2,GMS,Sat.1 Bayern,T • 1.002 15.06.–20.06.,20.06.2022,NaN,40 %,9 %,20 %,5 %,2 %,9 %,8 %,BP 1 % Sonst. 6 %
3,Forsa,RTL/n-tv,T • 1.049 23.05.–03.06.,07.06.2022,NaN,40 %,9 %,20 %,6 %,1 %,10 %,7 %,7 %
4,Forsa,VNP,T • 1.235 27.04.–13.05.,24.05.2022,NaN,39 %,10 %,20 %,5 %,2 %,11 %,6 %,7 %


In [20]:
parties = {
    'csu': make_dist(0.40, 0.02), 
    'spd': make_dist(0.08, 0.02), 
    'gruen': make_dist(0.18, 0.02), 
    'fdp': make_dist(0.06, 0.02), 
    'linke': make_dist(0.02, 0.02), 
    "fw": make_dist(0.10, 0.02),
    'afd': make_dist(0.11, 0.02)
}

def get_prediction():
    result = {}
    total = 0
    for name, p in parties.items():
        result[name] = p.rvs(1)[0]
        if result[name] < 0.05:
            result[name] = 0
        total += result[name]
    result = {name: p / total for name, p in result.items()}
    return result

def predictions():
    for _ in range(1000):
        yield get_prediction()

get_prediction()

{'csu': 0.41598421535187935,
 'spd': 0.06913879898993365,
 'gruen': 0.18920163587495611,
 'fdp': 0.06393658144954247,
 'linke': 0.0,
 'fw': 0.12891479934770284,
 'afd': 0.13282396898598559}

In [28]:
count = 0
count_csu = 0

outside = {}

for prediction in predictions():
    if prediction['csu'] > 0.5:
        count_csu += 1
    if prediction['csu'] + prediction['fw'] > 0.5:
        count += 1
    for name in prediction.keys():
        if prediction[name] == 0:
            if name not in outside:
                outside[name] = 0
            outside[name] += 1


print(outside)
print(count_csu / 10)
print(count / 10)

{'linke': 922, 'spd': 43, 'fdp': 324, 'fw': 2}
2.3
89.9


# Bundestag

In [30]:
parties = {
    'union': make_dist(0.28, 0.02), 
    'spd': make_dist(0.18, 0.02), 
    'gruen': make_dist(0.20, 0.02), 
    'fdp': make_dist(0.06, 0.02), 
    'linke': make_dist(0.05, 0.02), 
    'afd': make_dist(0.14, 0.02)
}

def get_prediction():
    result = {}
    total = 0
    for name, p in parties.items():
        result[name] = p.rvs(1)[0]
        if result[name] < 0.05:
            result[name] = 0
        total += result[name]
    result = {name: p / total for name, p in result.items()}
    return result

def predictions():
    for _ in range(1000):
        yield get_prediction()

get_prediction()

{'union': 0.34950711288085734,
 'spd': 0.22139534623551066,
 'gruen': 0.25541057707754816,
 'fdp': 0.0,
 'linke': 0.0,
 'afd': 0.17368696380608387}

In [34]:
count_rg = 0
count_ur = 0
count_ug = 0

outside = {}

for prediction in predictions():
    if prediction['spd'] + prediction['gruen'] > 0.5:
        count_rg += 1
    elif prediction['spd'] + prediction['union'] > 0.5:
        count_ur += 1
    elif prediction['union'] + prediction['gruen'] > 0.5:
        count_ug += 1

    for name in prediction.keys():
        if prediction[name] == 0:
            if name not in outside:
                outside[name] = 0
            outside[name] += 1


print(outside)
print(count_rg / 10)
print(count_ur / 10)
print(count_ug / 10)

{'linke': 549, 'fdp': 349}
3.9
70.3
19.4
